# CrewAI + MLflow Integration - Simple Travel Recommendation

This notebook demonstrates a basic integration between CrewAI and MLflow for tracking AI agent workflows. Creates a simple travel recommendation system that suggests the best city for photography travel and provide 5-day photography itinerary.

## Overview
- **CrewAI**: Framework for orchestrating AI agents
- **MLflow**: Experiment tracking and model management
- **Ollama**: Local LLM serving (Llama 3.2:3b model)

In [1]:
# Import necessary libraries for CrewAI and MLflow integration

from crewai import Agent, Crew, Task, LLM
from textwrap import dedent
import mlflow

# MLflow configuration

In [2]:
# Configure MLflow for automatic logging of CrewAI workflows
mlflow.crewai.autolog()

# Set MLflow tracking server (assumes MLflow server running locally on port 5000)
mlflow.set_tracking_uri("http://localhost:5000")

# Create/set experiment for organizing related runs
mlflow.set_experiment("CrewAI")

<Experiment: artifact_location='mlflow-artifacts:/267406013908563151', creation_time=1757424253324, experiment_id='267406013908563151', last_update_time=1757424253324, lifecycle_stage='active', name='CrewAI', tags={}>

## LLM Configuration

In [ ]:
# Define the local LLM model to use
LLM_TEXT_MODEL = "llama3.2:3b"

# Configure LLM connection to local Ollama server
llm = LLM(
    model=f"ollama/{LLM_TEXT_MODEL}", 
    base_url="http://localhost:11434",
    temperature=0.1,
    timeout=3000,
)

## Agent Definition

In [4]:
# Agents without tools (using only LLM knowledge)
city_selector = Agent(
    role="Travel Analyst",
    goal="Select best city based on given criteria",
    backstory="Expert travel consultant with global destination knowledge",
    llm=llm,
    verbose=True,
    max_iter=1,  # Single iteration only
    allow_delegation=False,
)

# Agent 2: Itinerary Planner
itinerary_planner = Agent(
    role="Photography Itinerary Specialist",
    goal="Create detailed photography itinerary for the recommended city",
    backstory=dedent("""
        Professional photography tour guide with extensive experience in planning 
        photo-focused travel itineraries. Expert in identifying the best times, 
        locations, and techniques for capturing stunning photographs in various cities.
    """),
    llm=llm,
    verbose=True,
    max_iter=1,
    allow_delegation=False,
)

## Task Execution

In [5]:
# Define the travel recommendation task
# Task 1: City Selection
task1 = Task(
    description=dedent("""
        Recommend the best city from Tokyo, Paris, New Delhi for photography travel 
        in August 2025 from California. Consider:
        - Weather conditions in August
        - Travel costs from California
        - Photography opportunities (landmarks, culture, nature)
        - Tourist crowds and accessibility
        
        Provide your recommendation with detailed reasoning.
    """),
    agent=city_selector,
    expected_output="Clear city recommendation with detailed reasoning covering weather, costs, and photography opportunities"
)

# Task 2: Itinerary Planning (depends on Task 1 output)
task2 = Task(
    description=dedent("""
        Based on the city recommendation from the Travel Analyst, create a detailed 
        5-day photography itinerary that includes:
        - Daily schedule with specific photography locations
        - Best times for golden hour and blue hour shots
        - Recommended photography equipment/settings
        - Local photography tips and cultural considerations
        - Transportation recommendations between photo spots
        - Alternative indoor locations for bad weather
        
        Focus on maximizing photographic opportunities while considering practical logistics.
    """),
    agent=itinerary_planner,
    expected_output="Comprehensive 5-day photography itinerary with specific locations, timing, and practical tips",
    context=[task1]  # This task depends on task1's output
)

## Task Execution

In [6]:
# Create crew with both agents and tasks
crew = Crew(
    agents=[city_selector, itinerary_planner], 
    tasks=[task1, task2], 
    verbose=True
)

# MLflow will automatically track this execution due to autolog()
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: dd7aa4f6-e564-46b9-97bc-db1ec4652bea                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Analyst                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Recommend the best city from Tokyo, Paris, New Delhi for photography travel                                    │
│  in August 2025 from California. Consider:                                                                      │
│  - Weather conditions in August                                                                                 │
│  - Travel costs from California                                                                                 │
│  - Photography opportunities (landmarks, culture, nature)                                                       │
│  - Tourist crowds and accessibility                                                                             │
│                                                                                                                 │
│  Provide your recommendation with detailed reasoning.                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Analyst                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Based on these factors, I recommend Tokyo for photography travel from California in August 2025. While Paris   │
│  is a more iconic destination, Tokyo's unique blend of traditional and modern architecture, vibrant street      │
│  life, and stunning landmarks make it an ideal choice for photographers. Additionally, the city's               │
│  well-organized infrastructure and convenient public transportation system minimize the impact of tourist       │
│  crowds.                                                                                                        │
│                                                                                                                 │
│  While New Delhi offers incredible cultural and historical photography opportunities, its hot and humid         │
│  weather in August may be challenging for some photographers. The high travel costs from California also make   │
│  Tokyo a more accessible option.                                                                                │
│                                                                                                                 │
│  To maximize your photography experience in Tokyo, consider visiting popular landmarks early in the morning or  │
│  later in the evening to avoid peak tourist hours. Additionally, explore lesser-known neighborhoods like        │
│  Shimokitazawa or Daikanyama for unique street life and architecture photography opportunities.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 44829802-592c-44d3-8559-6ec1b8c24f4f                                                                     │
│  Agent: Travel Analyst                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Photography Itinerary Specialist                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Based on the city recommendation from the Travel Analyst, create a detailed                                    │
│  5-day photography itinerary that includes:                                                                     │
│  - Daily schedule with specific photography locations                                                           │
│  - Best times for golden hour and blue hour shots                                                               │
│  - Recommended photography equipment/settings                                                                   │
│  - Local photography tips and cultural considerations                                                           │
│  - Transportation recommendations between photo spots                                                           │
│  - Alternative indoor locations for bad weather                                                                 │
│                                                                                                                 │
│  Focus on maximizing photographic opportunities while considering practical logistics.                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.APIConnectionError: OllamaException - litellm.Timeout: Connection timed out after 300.0         │
│  seconds.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: fd810b0b-e3db-4081-8fc0-c8c09d2ed14e                                                                     │
│  Agent: Photography Itinerary Specialist                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: dd7aa4f6-e564-46b9-97bc-db1ec4652bea                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

APIConnectionError: litellm.APIConnectionError: OllamaException - litellm.Timeout: Connection timed out after 300.0 seconds.

Trace(trace_id=tr-39177ceba8e7d1fbd723a603f5472e60)

## Results Display

In [ ]:
# Display the result in formatted markdown for better readability
from IPython.display import Markdown, display

# Convert result to string and display as formatted markdown
display(Markdown(str(result)))

## Next Steps and Enhancements

After running this notebook, explore these improvements and monitoring capabilities:

### 🔍 **Monitoring & Analysis**
1. **MLflow UI Dashboard**: Visit http://localhost:5000 to view tracked experiments
   - Compare runs with different parameters
   - Analyze agent execution times and token usage
   - Review conversation logs and outputs

2. **Performance Metrics**: Monitor key indicators
   - Task completion time
   - Token consumption
   - Success/failure rates across runs

### 🛠️ **Parameter Optimization**
1. **LLM Configuration Tuning**:
   - Experiment with temperature values (0.1 - 0.9)
   - Adjust max_iter for more complex reasoning
   - Test different Ollama models (llama3.1:8b, mistral, etc.)

2. **Agent Behavior Refinement**:
   - Modify backstory and role descriptions
   - Enable/disable delegation for multi-agent scenarios

### 🚀 **Advanced Features**
1. **Memory Integration**: Add persistent conversation memory
   ```python
   from crewai.memory import LongTermMemory
   crew = Crew(memory=LongTermMemory())
   ```

2. **Web Tools Integration**: Enable real-time data fetching
   - Weather APIs for current conditions
   - Flight price comparisons
   - Current events and travel advisories

3. **Knowledge Base Enhancement**:
   - RAG (Retrieval Augmented Generation) with travel databases
   - Custom tool integration for specialized data sources
   - Vector embeddings for destination similarity matching

4. **Multi-Agent Workflows**:
   - Add weather specialist agent
   - Include budget analyst agent
   - Create itinerary planning agent

### 📊 **Experiment Ideas**
- Compare different travel seasons and destinations
- Test various prompt engineering techniques
- Benchmark against different LLM models
- A/B testing of agent configurations